In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
import pandas as pd
import numpy as np
import string, os

In [5]:
def load_doc(filename):
  file=open(filename,'r')
  text=file.read()
  file.close()
  return text

In [ ]:
in_filename='/content/drive/MyDrive/alice.txt'
doc=load_doc(in_filename)
print(doc[:200])

In [12]:
import string
def clean_doc(doc):
  doc=doc.replace('--',' ')
  tokens=doc.split()
  table=str.maketrans('','',string.punctuation)
  tokens=[w.translate(table) for w in tokens]
  tokens=[word for word in tokens if word.isalpha()]
  tokens=[word.lower() for word in tokens]
  return tokens

In [ ]:
tokens=clean_doc(doc)

In [23]:
length=50+1
sequences=list()
for i in range(length,len(tokens)):
  seq=tokens[i-length:i]
  line=' '.join(seq)
  sequences.append(line)

In [24]:
def save_doc(lines,filename):
  data='\n'.join(lines)
  file=open(filename,'w')
  file.write(data)
  file.close()

In [25]:
out_filename='/content/drive/MyDrive/alice_sequences.txt'
save_doc(sequences,out_filename)

In [ ]:
doc1=load_doc(out_filename)
lines=doc1.split('\n')


In [42]:

tokenizer=Tokenizer()
tokenizer.fit_on_texts(lines)
sequences=tokenizer.texts_to_sequences(lines)


In [ ]:
vocab_size=len(tokenizer.word_index)+1
vocab_size

In [56]:
from tensorflow.keras.utils import to_categorical
sequences1= np.array(sequences)
X, y = sequences1[:,:-1], sequences1[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [64]:
# Embedding layer accepts input as(input_dim,output_dim,input_length)
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=10)
 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            154350    
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 3087)              311787    
Total params: 617,037
Trainable params: 617,037
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
230/230 [==============================] - 52s 213ms/step - loss: 6.4376 - accuracy: 0.0608
Epoch 

In [69]:

from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from pickle import dump

In [68]:
model.save('/content/drive/MyDrive/model.h5')
# save the tokenizer
dump(tokenizer, open('/content/drive/MyDrive/tokenizer.pkl', 'wb'))

In [ ]:
model = load_model('/content/drive/MyDrive/model.h5')

# load the tokenizer
tokenizer = load(open('/content/drive/MyDrive/tokenizer.pkl', 'rb'))

In [ ]:
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
  result = list()
  in_text = seed_text
  # generate a fixed number of words
  for _ in range(n_words):
      # encode the text as integer
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    # truncate sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    # predict probabilities for each word
    yhat = model.predict(encoded, verbose=0)
    # print(yhat,type(yhat),len(yhat))
    yhat=np.argmax(yhat,axis=1)
    # print(yhat)
    # map predicted word index to word
    out_word = ''
    for word, index in tokenizer.word_index.items():
      if index == yhat:
        out_word = word
        break
    # append to input
    in_text += ' ' + out_word
    result.append(out_word)
  return ' '.join(result)
 
seq_length = len(lines[0].split()) - 1
model = load_model('/content/drive/MyDrive/model.h5')

# load the tokenizer
tokenizer = load(open('/content/drive/MyDrive/tokenizer.pkl', 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
# print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

In [ ]:
seq_length = len(lines[0].split()) - 1
model = load_model('/content/drive/MyDrive/model.h5')

# load the tokenizer
tokenizer = load(open('/content/drive/MyDrive/tokenizer.pkl', 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
# print(seed_text + '\n')

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

In [82]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
# define model
inputs1 = Input(shape=(3, 1))
lstm1 = LSTM(1, return_sequences=True)(inputs1)
model = Model(inputs=inputs1, outputs=lstm1)
# define input data
data = array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
print(model.predict(data))

[[[0.01474419]
  [0.04063214]
  [0.07437418]]]
